In [ ]:
import os
import pandas as pd
import re

# Define the base directory where your subfolders are located
base_dir = "/media/carol/Data/Documents/Emo_rec/NewMel/IEMO_Mel_6"

# Initialize a dictionary to store data for the table
data = {}

# Traverse the base directory
for version_folder in os.listdir(base_dir):
    version_path = os.path.join(base_dir, version_folder)
    if os.path.isdir(version_path):
        data[version_folder] = {}
        
        for num_folder in range(10):  # Expected subfolders are numbered 0-9
            num_folder_path = os.path.join(version_path, str(num_folder))

            if os.path.exists(num_folder_path):
                header_file = os.path.join(num_folder_path, "header.txt")
                
                if os.path.exists(header_file):
                    # Read the header file
                    with open(header_file, "r") as f:
                        content = f.read()

                    # Extract Test UAR and Test Accuracy using regex
                    test_uar_match = re.search(r"test_uar\':\s([0-9\.]+)", content)
                    test_accuracy_match = re.search(r"test_accuracy\':\s([0-9\.]+)", content)

                    test_uar = float(test_uar_match.group(1)) if test_uar_match else None
                    test_accuracy = float(test_accuracy_match.group(1)) if test_accuracy_match else None

                    # Add to the dictionary
                    data[version_folder][str(num_folder)] = {
                        "Test UAR": test_uar,
                        "Test Accuracy": test_accuracy,
                    }
                else:
                    data[version_folder][str(num_folder)] = {
                        "Test UAR": None,
                        "Test Accuracy": None,
                    }
            else:
                data[version_folder][str(num_folder)] = {
                    "Test UAR": None,
                    "Test Accuracy": None,
                }

# Convert the dictionary to a DataFrame for better visualization
table_data = []
for version, num_data in data.items():
    for num, metrics in num_data.items():
        table_data.append({
            "Version": version,
            "Number": num,
            **metrics
        })

df = pd.DataFrame(table_data)

# Pivot the DataFrame for the desired structure
pivot_table = df.pivot(index="Version", columns="Number", values=["Test UAR", "Test Accuracy"])

# Flatten the MultiIndex for cleaner column names
pivot_table.columns = [f"{metric}_{num}" for metric, num in pivot_table.columns]
pivot_table.reset_index(inplace=True)

# Save or display the table
output_file = "metrics_table.csv"
pivot_table.to_csv(output_file, index=False)

# import ace_tools as tools; tools.display_dataframe_to_user("Metrics Table", pivot_table)

print(f"Metrics table saved to {output_file}")


ModuleNotFoundError: No module named 'ace_tools'